In [71]:
import numpy as np
from numpy import linalg as LA  # Notar que importamos linalg con nickname LA

In [72]:
np.random.seed(1)
c1 = 2*(np.random.rand(3,1)-0.5) # c1 y c2 son dos vectores 3x1 con elementos aleatorios en el 
c2 = 2*(np.random.rand(3,1)-0.5) # rango (-1,1)

print(np.dot(np.transpose(c1),c2)/(LA.norm(c1)*LA.norm(c2)))

# EL producto escalar entre c1 y c2 es c1·c2 = |c1||c2|cos(theta), entonces, 
# c1·c2/(|c1||c2|) = cos(theta), por lo que si este numero NO es 1 ni -1, sabemos que los vectores c1 y c2 no son linealmente independientes.
# Además, habiendo generado c1 y c2 al azar, las chances de que sean linealmente dependientes son minusculas.

[[0.4484599]]


In [73]:
d1 = 2*(np.random.rand()-0.5) # d1 y d2 son escalares en el rango (-1,1)
d2 = 2*(np.random.rand()-0.5)
c3 = d1*c1 + d2*c2  # c3, por construccion, es linealmente dependiente de c1 y c2.
print(c1)
print(c2)
print(c3) # Sin embargo eso NO es obvio a simple vista

[[-0.16595599]
 [ 0.44064899]
 [-0.99977125]]
[[-0.39533485]
 [-0.70648822]
 [-0.81532281]]
[[ 0.22624445]
 [-0.05827919]
 [ 0.87917177]]


In [74]:
A = np.concatenate((c1, c2, c3), axis=1) # construimos la matriz A con columnas c1, c2 y c3
print(A.shape)
print(A)  # chequear que, efectivamente, las columnas de A son c1, c2 y c3
LA.matrix_rank(A)  # EL rango de la matriz es el numero de columnas linealmente independientes

(3, 3)
[[-0.16595599 -0.39533485  0.22624445]
 [ 0.44064899 -0.70648822 -0.05827919]
 [-0.99977125 -0.81532281  0.87917177]]


2

In [75]:
# Construimos explicitamente A de modo que una de sus 3 columnas es linealmente dependiente
# de las otras dos. Ahora hagamos de cuenta que no sabemos esto. Por cierto que, dado
# que generamos las componentes de manera aleatoria, la dependencia linal no es obvia
# a simple vista, pero por construccion sabemos que esta ahi.
#
# Entonces, recordemos:
# 1) c1 y c2 son linealmente independiente. 
# 2) c3 es linealmente Dependiente de {c1, c2}
# 3) Como los coeficientes fueron generados al azar, con probabilidad 0.99999999999.....
#    c3 es lineamente independiente de c1 y c2 como vectores individuales, PERO ESTA EN EL PLANO GENERADO POR c1 y c2 (GeoGebra)

# 4) Por lo tanto, con altisima probabilidad, los pares {c1, c3} y {c2, c3} tambien son linealmente idependientes (chequearlo):

print(np.dot(np.transpose(c1),c2)/(LA.norm(c1)*LA.norm(c2)))
print(np.dot(np.transpose(c1),c3)/(LA.norm(c1)*LA.norm(c3)))
print(np.dot(np.transpose(c2),c3)/(LA.norm(c2)*LA.norm(c3)))


# 5) Sin embargo, la terna {c1, c2, c3} es linealmente dependiente. (comprobarlo en Geogebra)


# 6) Esto implica que el SPAN de {c1, c2, c3} es un plano, no es R3.



[[0.4484599]]
[[-0.93723406]]
[[-0.73198179]]


In [76]:
np.linalg.det(A)  # Una manera automatica de chequear si las columnas son linealmente
                  # dependientes es calcular el determinante. Si da cero son linealmente
                  # dependients. Ver el video de determinantes de 3Blue1Brown y entender  
                  # el significado geometrico del determinante (como se
                  # transforman los volumenes) para entender por que si son lin dep el det
                  # tiene que ser cero.
            
                  # Pero, aunque el determinante de A sea cero, y por lo tanto no podamos
                  # calcular la inversa de A, el sistema A x = b puede
                  # tener solucion si justo b pertence al subespacio spanned por las columnas
                  # de A. Vamos a averiguarlo con el metodo de Gram-Schmidt

-1.44283439087718e-17

In [77]:
# Como estamos jugando a que no sabemos como se construyo A, extraigamos los vectores columnas y,
# a través del proceso de Gram-Schmidt, obtengamos con una base ortonormal
# del subespacio generado por el span de las columnas de A
c1A = A[:, [0]]  # columna 1 de la matriz A 
print(c1A.shape)
print(c1A)
c2A = A[:, [1]]  # columna 2 de la matriz A 
print(c2A.shape)
print(c2A)
c3A = A[:, [2]]  # columna 3 de la matriz A
print(c3A.shape)
print(c3A)

(3, 1)
[[-0.16595599]
 [ 0.44064899]
 [-0.99977125]]
(3, 1)
[[-0.39533485]
 [-0.70648822]
 [-0.81532281]]
(3, 1)
[[ 0.22624445]
 [-0.05827919]
 [ 0.87917177]]


In [78]:
n1A = c1A/LA.norm(c1A)          # dividiendo al vector por su magnitud obtenemos un vector
                                # en la misma direccion pero de magnitud (o "norma") 1.
print(n1A)
print(np.dot(np.transpose(n1A),n1A))  # chequeamos que su norma es 1 multiplicandolo
                                      # escalarmente por si mismo

[[-0.15017224]
 [ 0.39873973]
 [-0.90468498]]
[[1.]]


In [79]:
# Ya tenemos el vector n1A, que es proporcional a c1A, pero de magnitud 1.
# Ahora queremos construir el vector n2A.
# Recordedmos que n2A es tiene que ser ortonormal con n1A, y tal que el span de {n1A, n2A}
# sea el mismo que el de {c1A, c2A}
# El mecanismo es: 
# 1) calculamos la componente de c2A paralela a n1A, la llamamos P_c2A_n1A
P_c2A_n1A = np.dot(np.transpose(n1A),c2A)*n1A # la proyeccion del vector c2A sobre el vector n1A es
                                              # (c2A.n1A) n1A 
                                              # c2A.n1A es producto escalar de C2A por n1A,
                                              # que da la magnitud de la proyeccion de c2A 
                                              # sobre sobre n1A. Luego, al multiplicarlo por n1A,
                                              # el vector resultante apunta en la direccion de n1A
                                              # La formula general inluye una division por 
                                              # (n1A.n1A)
                                              # en este caso no es necesario porque (n1A.n1A) = 1
print(P_c2A_n1A)

[[-0.07737981]
 [ 0.2054601 ]
 [-0.46616038]]


In [80]:
# 2) Calculamos la componente de c2A perpendicular a n1A.
#    A dicha componente la llamamos P_c2A_perp_n1A (largo pero explicito... :))
# 2_a) Se calcula simplemente restandole a c2A la componente paralela a n1A que 
#      ya calculamos en el paso 1.
#      Naturalmente, P_c2A_perp_n1A tiene que ser perpendicular a n1A. 
P_c2A_perp_n1A = c2A - P_c2A_n1A  # la proyeccion del vector c2A perpendicular al vector n1A es
                                  # c2A - P_c2A_n1A
print(P_c2A_perp_n1A)
print(np.dot(np.transpose(P_c2A_perp_n1A),P_c2A_n1A)) # para chequear la perpendicularidad de
                                                      # P_c2A_perp_n1A y P_c2A_n1A
                                                      # calculamos su producto escalar

[[-0.31795505]
 [-0.91194832]
 [-0.34916243]]
[[-2.77555756e-17]]


In [81]:
print(c2A)                          # Chequamos que c2A = P_c2A_n1A + P_c2A_perp_n1A de dos maneras
print(P_c2A_n1A + P_c2A_perp_n1A)   # 1- Observando numericamente si coinciden
c2A == P_c2A_n1A + P_c2A_perp_n1A   # 2- Preguntandole a Python con "=="
                                    # (ojo con este ultimo metodo porque puede llegar a dar
                                    # False por errores que las maquinas siempre cometen)

[[-0.39533485]
 [-0.70648822]
 [-0.81532281]]
[[-0.39533485]
 [-0.70648822]
 [-0.81532281]]


array([[ True],
       [ True],
       [ True]])

In [82]:
# 3) Calculamos n2A (que tiene que tener norma 1) simplemente dividiendo a 
#    P_c2A_perp_n1A por su norma.
# 3_a) Chequeamos que el conjunto {n1A, n2A} es ortonormal
#
n2A = P_c2A_perp_n1A/LA.norm(P_c2A_perp_n1A) # Generamos n2A, que es el vector de norma = 1
print(n2A)                                          # perpendicular a n1A 
print(np.dot(np.transpose(n2A),n2A))                # El conjunto {n1A, n2A} es ortonormal y
print(np.dot(np.transpose(n1A),n2A))                # genera el subespacio de R3 en el que "viven"
                                                    # c1A y c2A

[[-0.30960615]
 [-0.88800228]
 [-0.33999408]]
[[1.]]
[[-5.55111512e-17]]


In [83]:
# 4) Queremos calcular la componente de c3A perpendicular a n1A y n2A (y por lo tanto,
#    tambien perpendicular a c1A y c2A)
# 4_a) Primero calculamos las componentes de c3A paraleleas a n1A y n2A.

P_c3A_n1A = np.dot(np.transpose(n1A),c3A)*n1A # Continuamos con el proceso de Gram-Schmidt con
P_c3A_n2A = np.dot(np.transpose(n2A),c3A)*n2A # el vector c3A. Ahora estamos calculando las
print(P_c3A_n1A)                              # proyecciones de c3A sobre n1A y n2A
print(P_c3A_n2A)                              

[[ 0.12803496]
 [-0.33996046]
 [ 0.77132298]]
[[0.09820949]
 [0.28168127]
 [0.10784878]]


In [84]:
# 4_b) Calculamos la componente de c3A perpendicular a n1A y n2A.
#      A esto lo hacemos simplemente restando de c3A las componentes paralelas a n1A y a n2A

P_c3A_perp_n1Ayn2A = c3A - P_c3A_n1A - P_c3A_n2A # Intentamos extraer la componente de c3A que
print(P_c3A_perp_n1Ayn2A)                        # NO esta en el subespacio spanned por 
                                                 # {n1A, n2A} pero nos da zero!
                                                 # Esto significa que c3A el linealmente
                                                 # dependiente de c1A y c2A 
                                                 #
                                                 # Por eso concluimos que el sistema
                                                 # A x = b
                                                 # solo tendra solucion si justo b pertenece
                                                 # al mismo subespacio.

[[5.55111512e-17]
 [5.55111512e-17]
 [4.16333634e-17]]


In [88]:
# Nuestro proposito es entender cuando la ecuacion A x = b va a tener solucion.
# Y que tipo de soluciones tiene
#
# Miremos el lado izquierdo de la ecuacion: A x
# Recordemos que A x, donde x es un vector 3x1 de coeficientes incognita,
# tiene el significado geometrico de una combinacion lineal de los vecores columna de A.
# Los coeficientes de esa combinacion son los componentes incognita de x.
# 
# Ahora miremos el lado derecho: b
# la unica manera de que la ecuacion A x = b tenga solucion es que b viva en el subespacio spanned
# por las columnas de A: C(A).
# Entonces, si generamos un b al azar, con probabilidad ~1 la ecuacion no va a tener solucion:
b = 2*(np.random.rand(3,1)-0.5)
x = LA.solve(A, b)  
print("Non-solution for LA.Solve:",x)
print(np.linalg.cond(A))
# numericamente esto aparece como un vector gigante

Non-solution for LA.Solve: [[2.12232182e+16]
 [1.04471875e+16]
 [3.38229624e+16]]
2.468512216589437e+16


In [53]:
# Chequeamos que, efectivamente, x NO es una solucion de la ecuacion A x = b, o de A x - b = 0
print(np.dot(A,x)-b)

[[0.20646505]
 [0.29736653]
 [0.16161097]]


In [54]:
# Supongamos entonces que b vive en el subespacio generado por las columnas de A. Como generamos un b asi?

bb1 = 2*(np.random.rand()-0.5) # bb1 y bb2 son escalares en el rango (-1,1)
bb2 = 2*(np.random.rand()-0.5)
b = bb1*n1A + bb2*n2A            # Por que puedo estar seguro de que esto vive en el subespacio
print(b)                       # generado por las columnas de A? Numericamente no resulta obvio.

[[ 0.12737715]
 [ 0.6726029 ]
 [-0.13416163]]


In [55]:
x = LA.solve(A, b)                   # Asi resolvemos la ecuacion "singular", pero
                                     # todavia no entendemos conceptualmente como
                                     # encontrar esta solucion
print(x)

[[4.5241335 ]
 [1.35432943]
 [6.24809948]]


In [56]:
# Chequeamos que, efectivamente, x es una solucion de la ecuacion A x = b, o de A x - b = 0
print(np.dot(A,x)-b)

[[2.77555756e-17]
 [0.00000000e+00]
 [2.77555756e-16]]


In [57]:
# Como vimos, si hubiera 2 soluciones distintas entre si, x1 y x2 , de la ecuacion
# A x = b, esto es, si A x1 = b y A x2 = b y x1 distinto de x2,
# Entonces A (x1 - x2) = 0, donde (x1 - x2) es distinto del vector 0.
# En otras palabras, existirian vectores "d" distintos de cero tales que
#                       A d = 0
# Si d es solucion de la ecuacion "homogenea", lambda*d, donde lambda es cualquier
# numero real, tambien lo es.
# Que significado geometrico tiene d?
# Las componentes de d son incognita, llamemoslas d1, d2 y d3.
#                        A d = d1*c1A + d2*c2A + d3* c3A = 0
# es decir, A d es una combinacion lineal de los vectores columna de A que da como resultado
# el vector 0.
# Pero que exista tal combinacion lineal (no todos cero) implica que, por ejemplo
# (si d3 es distinto de cero):
#                    c3A = -(d1/d3)*c1A - (d2/d3)*c2A 
# es decir, que la tercera columna sea combinacion linal de las primeras dos.
# Es decir, implica que los vectores columna no son linealmente independientes. 
# 
# Esto implica que 
#                 (d1/d3)*c1A + (d2/d3)*c2A + c3A = 0

In [58]:
# recuerden que en una de las primeras celdas, construimos la columna c3 de A asi:
# d1 = 2*(np.random.rand()-0.5) 
# d2 = 2*(np.random.rand()-0.5)
# c3 = d1*c1 + d2*c2  # c3, por construccion, es linealmente dependiente de c1 y c2.
# Esto significa que el vector
x_0 = np.array([[-d1],[-d2],[1]])
print(x_0)
# tiene que ser solucion de la ecuacion homogenea A x = 0. Probemos:
np.dot(A,x_0)

[[0.62747958]
 [0.30887855]
 [1.        ]]


array([[0.],
       [0.],
       [0.]])

In [59]:
# Entonces x_0 multiplicado por cualquier escalar tambien tiene que ser solucion:
lambd = 20*(np.random.rand()-0.5) # numero aleatorio entre -10 y 10
x_01 = lambd*x_0 # ojo que la palabra lambda esta protegida
np.dot(A,x_01)

array([[ 0.00000000e+00],
       [-2.22044605e-16],
       [-8.88178420e-16]])

In [60]:
# Por lo tanto la solucion general de A x = b es una solucion particular x mas 
# una solucion cualquiera de la ecuacion A x = 0 (que llamamos x_01 = lambd*x_0):
print(np.dot(A,x + x_01) - b)

[[ 2.77555756e-17]
 [-5.55111512e-16]
 [ 2.77555756e-16]]


In [61]:
# Notar que en este caso la matriz A fue construida a proposito de modo que
# conocemos como construir la solucion de A x = 0.
# En casos realistas esto no es asi. La semana que viene aprenderemos a resolverla
# en general.

In [62]:
'''Autovalores y Autovectores'''

'Autovalores y Autovectores'

In [63]:
n=2
A = np.random.randn(n,n)
print(A)

[[ 1.46210794 -2.06014071]
 [-0.3224172  -0.38405435]]


In [64]:
w, v = LA.eig(A) # ver https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eig.html
print(w.shape)
print(v.shape)
print(w) # w es (2,) ((n,) en general), y cada componente es un autovalor de A LEER que significa (2,)
print(v) # v es (2,2) ((n,n) en general), y cada columna es un autovector de A

(2,)
(2, 2)
[ 1.77040959 -0.69235601]
[[ 0.98898696  0.69110861]
 [-0.14800267  0.72275092]]


In [65]:
AutoV1_A = v[:, [0]]
AutoV2_A = v[:, [1]]
print(AutoV1_A)
print(LA.norm(AutoV1_A))
print(AutoV2_A)
print(LA.norm(AutoV2_A))
print(np.dot(A,AutoV1_A)/AutoV1_A) # comprobamos que AutoV1_A es un autovector de A
                                   # con autovalor w[0]. (division por vector es comp. a comp.)
print(np.dot(A,AutoV2_A)/AutoV2_A) 

[[ 0.98898696]
 [-0.14800267]]
1.0
[[0.69110861]
 [0.72275092]]
1.0
[[1.77040959]
 [1.77040959]]
[[-0.69235601]
 [-0.69235601]]


In [66]:
# Comprovar visualmente en Geogebra que no son en general ortogonales
print(np.dot(np.transpose(AutoV1_A),AutoV2_A))

[[0.57652834]]


In [67]:
# A = (A + A^T)/2 + (A - A^T)/2, (A + A^T)/2 es simetrica, (A - A^T)/2 antisimetrica
As = (A + np.transpose(A))/2
print(As)

[[ 1.46210794 -1.19127896]
 [-1.19127896 -0.38405435]]


In [68]:
ws, vs = LA.eig(As) 
print(ws) 
print(vs)

[ 2.04608497 -0.96803139]
[[ 0.89791573  0.4401674 ]
 [-0.4401674   0.89791573]]


In [69]:
AutoV1_As = vs[:, [0]]
AutoV2_As = vs[:, [1]]
print(AutoV1_As)
print(AutoV2_As)
print(np.dot(np.transpose(AutoV1_As),AutoV2_As)) # Los autovectores de una matriz simetrica
                                                 # son ortogonales entre si

[[ 0.89791573]
 [-0.4401674 ]]
[[0.4401674 ]
 [0.89791573]]
[[0.]]
